In [1]:
pip install openpyxl plotly yfinance matplotlib ta pmdarima optuna requests tensorflow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 40.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [ ]:
## установка talib, pandas_ta не работает с мая
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/
import talib

In [ ]:
import numpy as np
import pandas as pd
import requests
import talib
import statsmodels.api as sm
import ta as ta2
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ks_2samp
import ast
import tensorflow as tf
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, MaxPooling1D, Conv1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Первый датасет

In [ ]:
## данные с 1 января 2019 по 9 сентября 2024
news_data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSTXRo2VOPKprX-FrgiR-iXAIy7tdNUMdyuZc5CE0qffRV8XWMmPhA3O1diMDAvZ7wm2JI9k-G510Ip/pub?output=csv')
news_data = news_data[['newsDatetime', 'negative', 'positive', 'important', 'liked', 'disliked', 'comments', 'currencies']]
news_data.head(50)

In [ ]:
## преобразуем дату
news_data['newsDatetime'] = pd.to_datetime(news_data['newsDatetime'], format='mixed').dt.date
news_data

In [ ]:
news_data['newsDatetime'].max()

In [ ]:
news_data['newsDatetime'].min()

In [ ]:
news_data['liked'].value_counts()

In [ ]:
## ищем пропуски
news_data.isna().sum()

In [ ]:
## на данных видно что есть пропуски в currencies, так что удалим пропущенные значения
news_data.dropna(inplace=True)

# Получение датасетов по отдельным криптовалютам

In [ ]:
def create_new_dataset(crypto_name):
    ## берем данные где содержится название криптовалюты
    some_news_data = news_data[news_data['currencies'].str.contains(crypto_name)]
    some_news_data.reset_index(inplace=True)
    ## убираем уже не нужные столбцы
    some_news_data = some_news_data.drop(['currencies', 'index'], axis=1)
    ## считаем суммы для параметров
    some_news_data = some_news_data.groupby(by='newsDatetime', as_index=False). \
        agg({'negative': 'sum', 'positive': 'sum', 'important': 'sum',	'liked': 'sum',	'disliked': 'sum', 'comments': 'sum'})
    return some_news_data

In [ ]:
bitcoin_news_data = create_new_dataset('BTC')
bitcoin_news_data

In [ ]:
eth_news_data = create_new_dataset('ETH')
eth_news_data

In [ ]:
sol_news_data = create_new_dataset('SOL')
sol_news_data

In [ ]:
card_news_data = create_new_dataset('ADA')
card_news_data

In [ ]:
ripp_news_data = create_new_dataset('XRP')
ripp_news_data

In [ ]:
doge_news_data = create_new_dataset('DOGE')
doge_news_data

# Получение цены криптовалют с помощью API

In [ ]:
## функция для получения данных с Binance
def get_crypto_price(symbol, interval, days):
    api_key = 'ace6a59ff46446659af4e691fab88e22'
    api_url = f'https://api.twelvedata.com/time_series?symbol={symbol}&exchange=Binance&interval={interval}&outputsize={days}&apikey={api_key}'
    raw = requests.get(api_url).json()
    df = pd.DataFrame(raw['values'])
    df = df.iloc[::-1].reset_index(drop=True)
    return df

In [ ]:
## функция для перевода значений в числовой формат
def prices_to_numeric(some_prices):
    some_prices['open'] = pd.to_numeric(some_prices['open'])
    some_prices['high'] = pd.to_numeric(some_prices['high'])
    some_prices['low'] = pd.to_numeric(some_prices['low'])
    some_prices['close'] = pd.to_numeric(some_prices['close'])
    return some_prices

In [ ]:
bitcoin_prices = get_crypto_price('BTC/USD', '1day', 1431)
bitcoin_prices = bitcoin_prices[:1348]
bitcoin_prices['datetime'] = pd.to_datetime(bitcoin_prices['datetime'], format='mixed').dt.date
bitcoin_prices = prices_to_numeric(bitcoin_prices)
bitcoin_prices

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=bitcoin_prices['datetime'], y=bitcoin_prices['close'], mode='lines', name='VALUE'))
fig.update_layout(title='Визуализация цены биткоина', xaxis_title='Дата', yaxis_title='Значение')
fig.show()

In [ ]:
eth_prices = get_crypto_price('ETH/USD', '1day', 1428)
eth_prices = eth_prices[:1348]
eth_prices['datetime'] = pd.to_datetime(eth_prices['datetime'], format='mixed').dt.date
eth_prices = prices_to_numeric(eth_prices)
eth_prices

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=eth_prices['datetime'], y=eth_prices['close'], mode='lines', name='VALUE'))
fig.update_layout(title='Визуализация цены etherium', xaxis_title='Дата', yaxis_title='Значение')
fig.show()

In [ ]:
sol_prices = get_crypto_price('SOL/USD', '1day', 1431)
sol_prices = sol_prices[:1348]
sol_prices['datetime'] = pd.to_datetime(sol_prices['datetime'], format='mixed').dt.date
sol_prices = prices_to_numeric(eth_prices)
sol_prices

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sol_prices['datetime'], y=sol_prices['close'], mode='lines', name='VALUE'))
fig.update_layout(title='Визуализация цены solana', xaxis_title='Дата', yaxis_title='Значение')
fig.show()

In [ ]:
doge_prices = get_crypto_price('DOGE/USD', '1day', 1431)
doge_prices = doge_prices[:1348]
doge_prices['datetime'] = pd.to_datetime(doge_prices['datetime'], format='mixed').dt.date
doge_prices = prices_to_numeric(eth_prices)
doge_prices

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=doge_prices['datetime'], y=doge_prices['close'], mode='lines', name='VALUE'))
fig.update_layout(title='Визуализация цены dogecoin', xaxis_title='Дата', yaxis_title='Значение')
fig.show()

In [ ]:
ripp_prices = get_crypto_price('XRP/USD', '1day', 1431)
ripp_prices = doge_prices[:1348]
ripp_prices['datetime'] = pd.to_datetime(ripp_prices['datetime'], format='mixed').dt.date
ripp_prices = prices_to_numeric(ripp_prices)
ripp_prices

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ripp_prices['datetime'], y=ripp_prices['close'], mode='lines', name='VALUE'))
fig.update_layout(title='Визуализация цены ripple', xaxis_title='Дата', yaxis_title='Значение')
fig.show()

In [ ]:
card_prices = get_crypto_price('ADA/USD', '1day', 1431)
card_prices = card_prices[:1348]
card_prices['datetime'] = pd.to_datetime(card_prices['datetime'], format='mixed').dt.date
card_prices = prices_to_numeric(card_prices)
card_prices

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=card_prices['datetime'], y=card_prices['close'], mode='lines', name='VALUE'))
fig.update_layout(title='Визуализация цены cardano', xaxis_title='Дата', yaxis_title='Значение')
fig.show()

# Добавление новых признаков

In [ ]:
def adding_new_features(some_prices):
    # Индекс относительной силы (RSI)
    some_prices['rsi'] = talib.RSI(some_prices['close'], timeperiod=14)

    # MACD (Moving Average Convergence Divergence)
    macd, signal_line, histogram = talib.MACD(some_prices['close'], \
                    fastperiod=12,slowperiod=26,signalperiod=9)
    some_prices['macd'] = macd

    # Короткая и длинная скользящие средние
    some_prices['moving_average_short'] = talib.SMA(some_prices['close'], timeperiod=20)
    some_prices['moving_average_long'] = talib.SMA(some_prices['close'], timeperiod=50)

    # Полосы Боллинджера
    upper, middle, lower = talib.BBANDS(some_prices['close'],timeperiod=20,nbdevup=2,
    nbdevdn=2, matype=0)

    some_prices['bollinger_upper'] = upper
    some_prices['bollinger_lower'] = lower

    # Стохастический осциллятор
    slowk, slowd = talib.STOCH(some_prices['high'],some_prices['low'],some_prices['close'],
    fastk_period=14,slowk_period=3,slowk_matype=0, slowd_period=3,slowd_matype=0)

    some_prices['stoch_k'] = slowk
    some_prices['stoch_d'] = slowd

    # Средний истинный диапазон (ATR)
    some_prices['atr'] = talib.ATR( some_prices['high'], some_prices['low'],
        some_prices['close'],timeperiod=14)

    # Индекс среднего направленного движения (ADX)
    some_prices['adx'] = talib.ADX(some_prices['high'],some_prices['low'],some_prices['close'],
                                  timeperiod=14)

    # Индикатор Вильямса %R
    some_prices['williams_r'] = talib.WILLR(some_prices['high'],some_prices['low'],
        some_prices['close'],timeperiod=14)

    # Индекс товарного канала (CCI)
    some_prices['cci'] = talib.CCI(some_prices['high'],some_prices['low'],some_prices['close'],
                                   timeperiod=20)


    # Заполнение пропущенных значений
    some_prices.fillna(0, inplace=True)

    return some_prices

In [ ]:
adding_new_features(bitcoin_prices)

In [ ]:
## сделаем одну таблицу со всеми признаками
bitcoin_data = bitcoin_news_data.merge(bitcoin_prices, how='inner', left_on='newsDatetime', right_on='datetime')
## уберем повторяющийся столбец
bitcoin_data.drop(['datetime'], inplace=True, axis=1)
bitcoin_data

In [ ]:
## надо построить график

In [ ]:
adding_new_features(eth_prices)

In [ ]:
## сделаем одну таблицу со всеми признаками
eth_data = eth_news_data.merge(eth_prices, how='inner', left_on='newsDatetime', right_on='datetime')
## уберем повторяющийся столбец
eth_data.drop(['datetime'], inplace=True, axis=1)
eth_data

In [ ]:
adding_new_features(sol_prices)

In [ ]:
## сделаем одну таблицу со всеми признаками
sol_data = sol_news_data.merge(sol_prices, how='inner', left_on='newsDatetime', right_on='datetime')
## уберем повторяющийся столбец
sol_data.drop(['datetime'], inplace=True, axis=1)
sol_data

In [ ]:
adding_new_features(doge_prices)

In [ ]:
## сделаем одну таблицу со всеми признаками
doge_data = doge_news_data.merge(doge_prices, how='inner', left_on='newsDatetime', right_on='datetime')
## уберем повторяющийся столбец
doge_data.drop(['datetime'], inplace=True, axis=1)
doge_data

In [ ]:
adding_new_features(ripp_prices)

In [ ]:
## сделаем одну таблицу со всеми признаками
ripp_data = ripp_news_data.merge(ripp_prices, how='inner', left_on='newsDatetime', right_on='datetime')
## уберем повторяющийся столбец
ripp_data.drop(['datetime'], inplace=True, axis=1)
ripp_data

In [ ]:
adding_new_features(card_prices)

In [ ]:
## сделаем одну таблицу со всеми признаками
card_data = card_news_data.merge(card_prices, how='inner', left_on='newsDatetime', right_on='datetime')
## уберем повторяющийся столбец
card_data.drop(['datetime'], inplace=True, axis=1)
card_data

# Визуализация признаков временных рядов

In [ ]:
def visualize_with_plotly(technical_data):
    fig = go.Figure()

    # negative
    fig.add_trace(go.Scatter(x=technical_data['newsDatetime'], y=technical_data['negative'], mode='lines', name='NEGATIVE'))

    # positive
    fig.add_trace(go.Scatter(x=technical_data['newsDatetime'], y=technical_data['positive'], mode='lines', name='POSITIVE'))

    # important
    fig.add_trace(go.Scatter(x=technical_data['newsDatetime'], y=technical_data['important'], mode='lines', name='IMPORTANT'))


    # liked
    fig.add_trace(go.Scatter(x=technical_data['newsDatetime'], y=technical_data['liked'], mode='lines', name='LIKED'))

    # disliked
    fig.add_trace(go.Scatter(x=technical_data['newsDatetime'], y=technical_data['disliked'], mode='lines', name='DISLIKED'))

    # comments
    fig.add_trace(go.Scatter(x=technical_data['newsDatetime'], y=technical_data['comments'], mode='lines', name='COMMENTS'))

    fig.update_layout(title='Визуализация данных из соцсетей', xaxis_title='Дата', yaxis_title='Значение')
    fig.show()

In [ ]:
## для bitcoin
visualize_with_plotly(bitcoin_data)

In [ ]:
## для etherium
visualize_with_plotly(eth_data)

In [ ]:
## для solana
visualize_with_plotly(sol_data)

In [ ]:
## для ripple
visualize_with_plotly(ripp_data)

In [ ]:
## для dogecoin
visualize_with_plotly(doge_data)

In [ ]:
## для cardano
visualize_with_plotly(card_data)

# Удаление выбросов(для числовых данных)

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

def data_clean(some_data):
    some_data_clean = some_data.copy()
    columns_to_normalize = ['close', 'negative', 'positive']
    if columns_to_normalize:
        for col in columns_to_normalize:
            some_data_clean  = remove_outliers(some_data_clean, col)
    some_data_clean.reset_index(inplace=True, drop=True)
    return some_data_clean


In [ ]:
def remove_zero_sum(some_data):
    sum_values = some_data['negative'] + some_data['positive']	+ some_data['important'] \
        + some_data['liked'] + some_data['disliked'] + some_data['comments']
    some_data = some_data[sum_values != 0]
    some_data.reset_index(drop=True, inplace=True)
    return some_data

In [ ]:
bitcoin_data_clean = data_clean(bitcoin_data)
bitcoin_data_clean = remove_zero_sum(bitcoin_data_clean)
bitcoin_data_clean['diff'] = bitcoin_data_clean['positive'] - bitcoin_data_clean['negative']
bitcoin_data_clean

In [ ]:
eth_data_clean = data_clean(eth_data)
eth_data_clean = remove_zero_sum(eth_data_clean)
eth_data_clean['diff'] = eth_data_clean['positive'] - eth_data_clean['negative']
eth_data_clean

In [ ]:
sol_data_clean = data_clean(sol_data)
sol_data_clean = remove_zero_sum(sol_data_clean)
sol_data_clean['diff'] = sol_data_clean['positive'] - sol_data_clean['negative']
sol_data_clean

In [ ]:
doge_data_clean = data_clean(doge_data)
doge_data_clean = remove_zero_sum(doge_data_clean)
doge_data_clean['diff'] = doge_data_clean['positive'] - doge_data_clean['negative']
doge_data_clean

In [ ]:
ripp_data_clean = data_clean(ripp_data)
ripp_data_clean = remove_zero_sum(ripp_data_clean)
ripp_data_clean['diff'] = ripp_data_clean['positive'] - ripp_data_clean['negative']
ripp_data_clean

In [ ]:
card_data_clean = data_clean(card_data)
card_data_clean = remove_zero_sum(card_data_clean)
card_data_clean['diff'] = card_data_clean['positive'] - card_data_clean['negative']
card_data_clean

# Второй датасет

In [ ]:
data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQDvTddWwdOUSuPtsn94kN_nuaIBa88H7nsCdKJsvrdz2D_cQk6i7HYEg46RCYjMu6gnephSqQqodq9/pub?output=csv').drop(['source', 'text', 'url'], axis=1)
data

In [ ]:
data['date'] = pd.to_datetime(data['date'], format='mixed').dt.date
data

In [ ]:
## функция для перевода класса в числовой формат
def class_to_num(x):
    if x == 'negative':
        return -1
    elif x == 'positive':
        return 1
    elif x == 'neutral':
        return 0

In [ ]:
## парсинг столбца sentiment
data['class'] = data['sentiment'].apply(lambda x: ast.literal_eval(x)['class']).apply(class_to_num)
data['polarity'] = data['sentiment'].apply(lambda x: ast.literal_eval(x)['polarity'])
data['subjectivity'] = data['sentiment'].apply(lambda x: ast.literal_eval(x)['subjectivity'])
## удаление столбца sentiment (все данные уже были извлечены)
data = data.drop(['sentiment'], axis=1)
data

In [ ]:
## создание датасета с новостями про биткоин
bitcoin_news = data[data['subject'].str.contains('bitcoin')]
bitcoin_news = bitcoin_news.reset_index(drop=True)
## введем индекс, который будет произведением типа новости(хорошая, плохая и нейтральная) и субъективности новости
bitcoin_news['index'] = bitcoin_news['class'] * bitcoin_news['subjectivity']
## найдем среднее значение полученного индекса и суммы типа новостей(т.е разницы между хорошими и плохими новостями)
bitcoin_news = bitcoin_news.groupby("date", as_index=False).agg({'index': 'mean', 'class': 'sum'})
bitcoin_news.columns = list(map(''.join, bitcoin_news.columns.values))
bitcoin_news.rename(columns={'index':'mean_index', 'class': 'class_diff'}, inplace=True)
bitcoin_news

In [ ]:
## сделаем новый датасет со всеми признаками
bitcoin_data_new = bitcoin_news.merge(bitcoin_prices, how='inner', left_on='date', right_on='datetime')
## уберем повторяющийся столбец
bitcoin_data_new.drop(['datetime'], inplace=True, axis=1)
bitcoin_data_new

In [ ]:
## создание датасета с новостями про etherium
eth_news = data[data['subject'].str.contains('ethereum')]
eth_news = eth_news.reset_index(drop=True)
## введем индекс, который будет произведением типа новости(хорошая, плохая и нейтральная) и субъективности новости
eth_news['index'] = eth_news['class'] * eth_news['subjectivity']
## найдем среднее значение полученного индекса и суммы типа новостей(т.е разницы между хорошими и плохими новостями)
eth_news = eth_news.groupby("date", as_index=False).agg({'index': 'mean', 'class': 'sum'})
eth_news.columns = list(map(''.join, eth_news.columns.values))
eth_news.rename(columns={'index':'mean_index', 'class': 'class_diff'}, inplace=True)
eth_news

In [ ]:
## сделаем новый датасет со всеми признаками
eth_data_new = eth_news.merge(eth_prices, how='inner', left_on='date', right_on='datetime')
## уберем повторяющийся столбец
eth_data_new.drop(['datetime'], inplace=True, axis=1)
eth_data_new

# Визуализация новых признаков

In [ ]:
bitcoin_prices

In [ ]:
# Визуализация с использованием Matplotlib и Seaborn
def visualize_with_matplotlib(technical_data):
    # Фигура и подграфики
    fig, axs = plt.subplots(4, 2, figsize=(14, 16))

    # Индикаторы RSI и MACD
    sns.lineplot(x='datetime', y='rsi', data=technical_data, ax=axs[0, 0])
    axs[0, 0].set_title('RSI (Индекс относительной силы)')

    sns.lineplot(x='datetime', y='macd', data=technical_data, ax=axs[0, 1])
    axs[0, 1].set_title('MACD (Конвергенция/дивергенция скользящих средних)')

    # Полосы Боллинджера
    sns.lineplot(x='datetime', y='bollinger_upper', data=technical_data, ax=axs[1, 0], label='Верхняя полоса')
    sns.lineplot(x='datetime', y='bollinger_lower', data=technical_data, ax=axs[1, 0], label='Нижняя полоса')
    axs[1, 0].set_title('Полосы Боллинджера')

    # Скользящие средние
    sns.lineplot(x='datetime', y='moving_average_short', data=technical_data, ax=axs[1, 1], label='Короткая СМА')
    sns.lineplot(x='datetime', y='moving_average_long', data=technical_data, ax=axs[1, 1], label='Длинная СМА')
    axs[1, 1].set_title('Скользящие средние')

    # Стохастический осциллятор K и D
    sns.lineplot(x='datetime', y='stoch_k', data=technical_data, ax=axs[2, 0], label='Стох. K')
    sns.lineplot(x='datetime', y='stoch_d', data=technical_data, ax=axs[2, 0], label='Стох. D')
    axs[2, 0].set_title('Стохастический осциллятор')

    # ATR (Средний истинный диапазон)
    sns.lineplot(x='datetime', y='atr', data=technical_data, ax=axs[2, 1])
    axs[2, 1].set_title('ATR (Средний истинный диапазон)')

    # ADX (Индекс среднего направленного движения)
    sns.lineplot(x='datetime', y='adx', data=technical_data, ax=axs[3, 0])
    axs[3, 0].set_title('ADX (Индекс среднего направленного движения)')

    # самая высокая цена за день
    sns.lineplot(x='datetime', y='high', data=technical_data, ax=axs[3, 1])
    axs[3, 1].set_title('Максимальная цена за день')

    plt.tight_layout()
    plt.show()


In [ ]:
visualize_with_matplotlib(bitcoin_prices)

In [ ]:
visualize_with_matplotlib(eth_prices)

In [ ]:
visualize_with_matplotlib(doge_prices)

In [ ]:
visualize_with_matplotlib(ripp_prices)

In [ ]:
visualize_with_matplotlib(sol_prices)

In [ ]:
visualize_with_matplotlib(card_prices)

# Нормализация данных

In [ ]:
def normalizer(some_data, train_split):
    some_data_mean = some_data.iloc[:train_split, 1:].mean()
    some_data_std = some_data.iloc[:train_split, 1:].std()

    some_data.iloc[:train_split, 1:] = (some_data.iloc[:train_split, 1:] - some_data_mean) / some_data_std
    return some_data, some_data_mean, some_data_std

In [ ]:
BITCOIN_train_split = 750
#bitcoin_data_clean, bitcoin_data_mean, bitcoin_data_std = normalizer(bitcoin_data_clean, BITCOIN_train_split)
bitcoin_data_clean

In [ ]:
ETH_train_split = 750
#eth_data_clean, eth_data_mean, eth_data_std = normalizer(eth_data_clean, ETH_train_split)
eth_data_clean

In [ ]:
SOL_train_split = 550
#sol_data_clean, sol_data_mean, sol_data_std = normalizer(sol_data_clean, SOL_train_split)
sol_data_clean

In [ ]:
DOGE_train_split = 480
#doge_data_clean, doge_data_mean, doge_data_std = normalizer(doge_data_clean, DOGE_train_split)
doge_data_clean

In [ ]:
RIPP_train_split = 650
#ripp_data_clean, ripp_data_mean, ripp_data_std = normalizer(ripp_data_clean, RIPP_train_split)
ripp_data_clean

In [ ]:
CARD_train_split = 700
#card_data_clean, card_data_mean, card_data_std = normalizer(card_data_clean, CARD_train_split)
card_data_clean

In [ ]:
#BITCOIN_train_split_new = 600
#bitcoin_data_new = normalizer(bitcoin_data_new, BITCOIN_train_split_new)
#bitcoin_data_new

In [ ]:
#ETH_train_split_new = 600
#eth_data_new = normalizer(eth_data_new, ETH_train_split_new)
#eth_data_new

In [ ]:
TRAIN_SPLIT = 800

# Model 1 Прогнозирование на основе одномерного временного ряда

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title, ax_n=False,  graph_count=1):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0
  if ax_n:
      plt.subplot(graph_count, 1, ax_n)
  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [ ]:
def baseline(history):
  return np.mean(history)

In [ ]:
def plot_train_history(history, title, ax_n=False, graph_count=1):
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(loss))
  if ax_n:
      plt.subplot(graph_count, 1, ax_n)
  else:
      plt.figure()
  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()
  return plt

In [ ]:
def use_first_model(some_data_clean, TRAIN_SPLIT):
    fig, axs = plt.subplots(6, 1, figsize=(15, 48))

    uni_data = some_data_clean['close']
    uni_data.index = some_data_clean['newsDatetime']
    uni_data = uni_data.values

    uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
    uni_train_std = uni_data[:TRAIN_SPLIT].std()

    uni_data = (uni_data-uni_train_mean)/uni_train_std


    univariate_past_history = 20
    univariate_future_target = 0

    x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                               univariate_past_history,
                                               univariate_future_target)
    x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                           univariate_past_history,
                                           univariate_future_target)

    show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example', ax_n=1, graph_count=6)

    show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,'Baseline Prediction Example', ax_n=2, graph_count=6)

    BATCH_SIZE = 64
    BUFFER_SIZE = 10000

    train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
    train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

    val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
    val_univariate = val_univariate.batch(BATCH_SIZE).repeat()


    simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
    ])

    simple_lstm_model.compile(optimizer='adam', loss='mae')

    #Проверим выход модели.
    # for x, y in val_univariate.take(1):
    #     print(simple_lstm_model.predict(x).shape)

    EVALUATION_INTERVAL = 200
    EPOCHS = 30

    single_step_history = simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                          steps_per_epoch=EVALUATION_INTERVAL,
                          validation_data=val_univariate, validation_steps=50)
    plot_train_history(single_step_history, 'Single Step Training and validation loss', ax_n=3,  graph_count=6)
    subplot_ind = 4
    for x, y in val_univariate.take(3):
        plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model', ax_n=subplot_ind, graph_count=6)
        subplot_ind += 1
    plot.show()

In [ ]:
use_first_model(bitcoin_data_clean, BITCOIN_train_split)

In [ ]:
use_first_model(eth_data_clean, ETH_train_split)

In [ ]:
use_first_model(sol_data_clean, SOL_train_split)

In [ ]:
use_first_model(doge_data_clean, DOGE_train_split)

In [ ]:
use_first_model(ripp_data_clean, RIPP_train_split)

In [ ]:
use_first_model(card_data_clean, CARD_train_split)

# Model 2

In [ ]:
def use_second_model(some_data_clean, TRAIN_SPLIT):

    fig, axs = plt.subplots(4, 1, figsize=(15, 32))

    uni_data = some_data_clean['close']
    uni_data.index = some_data_clean['newsDatetime']
    uni_data = uni_data.values

    uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
    uni_train_std = uni_data[:TRAIN_SPLIT].std()

    uni_data = (uni_data-uni_train_mean)/uni_train_std
    BATCH_SIZE = 64
    BUFFER_SIZE = 1000

    univariate_past_history = 20
    univariate_future_target = 0

    x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                               univariate_past_history,
                                               univariate_future_target)
    x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                           univariate_past_history,
                                           univariate_future_target)
    train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
    train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

    val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
    val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

    simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(16, return_sequences=True, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.LSTM(16, return_sequences=False),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
    ])

    #simple_lstm_model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(clipvalue=1.0), loss='mae')
    simple_lstm_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

    EVALUATION_INTERVAL = 200
    EPOCHS = 30

    single_step_history = simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                          steps_per_epoch=EVALUATION_INTERVAL,
                          validation_data=val_univariate, validation_steps=50)
    plot_train_history(single_step_history, 'Single Step Training and validation loss', ax_n=1, graph_count=4)

    subplot_ind = 2
    for x, y in val_univariate.take(3):
      plot = show_plot([x[0].numpy(), y[0].numpy(),
                        simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model', ax_n=subplot_ind, graph_count=4)
      subplot_ind += 1
    plot.show()

In [ ]:
use_second_model(bitcoin_data_clean, BITCOIN_train_split)

In [ ]:
use_second_model(eth_data_clean, ETH_train_split)

In [ ]:
use_second_model(sol_data_clean, SOL_train_split)

In [ ]:
use_second_model(ripp_data_clean, RIPP_train_split)

In [ ]:
use_second_model(doge_data_clean, DOGE_train_split)

In [ ]:
use_second_model(card_data_clean, CARD_train_split)

# Model 3 Прогнозирование на основе многомерного временного ряда

In [ ]:
price_features_considered = ['close', 'macd', 'moving_average_short', 'moving_average_long',
       'bollinger_upper', 'bollinger_lower', 'stoch_k', 'stoch_d', 'atr', 'adx', 'williams_r', 'cci']
news_features_considered = ['negative', 'positive', 'important', 'liked', 'disliked', 'comments', 'diff']



In [ ]:
bitcoin_data_clean[price_features_considered[:4]]

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)


def use_third_model(some_data_clean, TRAIN_SPLIT, features_considered):
    fig, axs = plt.subplots(4, 1, figsize=(15, 32))

    features = some_data_clean[features_considered]
    features.index = some_data_clean['newsDatetime']

    dataset = features.values
    data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
    data_std = dataset[:TRAIN_SPLIT].std(axis=0)

    dataset = (dataset-data_mean)/data_std
    dataset0 = features.values
    data_mean = dataset0[:TRAIN_SPLIT].mean(axis=0)
    data_std = dataset0[:TRAIN_SPLIT].std(axis=0)

    dataset = (dataset0-data_mean)/data_std

    dataset2 = dataset * data_std + data_mean

    EVALUATION_INTERVAL = 200
    EPOCHS = 30
    BATCH_SIZE = 64
    BUFFER_SIZE = 1000
    past_history = 30
    future_target = 1
    STEP = 1

    x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
    x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

    print ('Single window of past history')
    print (x_train_single[0])
    print ('\n Target to predict')
    print (y_train_single[0])


    train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
    train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

    val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
    val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

    single_step_model = tf.keras.models.Sequential()
    single_step_model.add(tf.keras.layers.LSTM(50, return_sequences=True, input_shape=x_train_single.shape[-2:]))
    single_step_model.add(tf.keras.layers.LSTM(50, return_sequences=False))  #, activation='relu'
    single_step_model.add(tf.keras.layers.Dense(25))
    single_step_model.add(tf.keras.layers.Dense(1))

    #single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')
    single_step_model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        clipnorm=None,
        clipvalue=None,
        global_clipnorm=None,
        name='adamax'), loss='mae')

    #Проверим нашу выборку и выведем кривые потерь на этапах обучения и проверки.

    # Раннее завершение обучения
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


    single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                                steps_per_epoch=EVALUATION_INTERVAL,
                                                validation_data=val_data_single,
                                                validation_steps=50,
                                               callbacks=[early_stopping])


    plot_train_history(single_step_history, 'Single Step Training and validation loss', ax_n=1, graph_count=4)


    subplot_ind = 2
    for x, y in val_data_single.take(3):
        plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                        single_step_model.predict(x)[0]], 0,
                       'Single Step Prediction', ax_n=subplot_ind, graph_count=4)
        subplot_ind += 1
        plot.show()

In [ ]:
use_third_model(bitcoin_data_clean, BITCOIN_train_split, ['close','macd','moving_average_short', 'stoch_k', 'atr'])

In [ ]:
use_third_model(eth_data_clean, ETH_train_split,  ['close','macd','moving_average_short', 'atr'])

In [ ]:
use_third_model(sol_data_clean, SOL_train_split, price_features_considered[:4])

In [ ]:
use_third_model(ripp_data_clean, RIPP_train_split, price_features_considered[:4])

In [ ]:
use_third_model(doge_data_clean, DOGE_train_split, ['close','macd','moving_average_short', 'stoch_k', 'atr'])

In [ ]:
use_third_model(card_data_clean, CARD_train_split, price_features_considered[:4])

# Model ARIMA

In [ ]:
!pip install numpy==1.26.4 pandas==2.2.2 --quiet


In [ ]:
import os

# INstalacion de la correcta version de numpy para funcionar con pmdarima
!pip install numpy==1.26.4 --quiet

# Reiniciamos la sesion de google colab
os.kill(os.getpid(), 9)

In [ ]:
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
def divides_into_samples(df):
    """Делит данные на обучающую, валидационную и тестовую выборки.
    """
    # 1. Определим признаки (X) и целевую переменную (y)
    X = df.drop('close', axis=1)
    y = df['close']

    # 2. Разделим данные на обучающую и тестовую выборки (80% обучающих, 20% тестовых)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 3. Разделим тестовую выборку на валидационную и тестовую (50% валидационных, 50% тестовых)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

    return X_train, X_test, X_val, y_train, y_test, y_val

def divides_into_samples_2(df):

    X = df.drop('close', axis=1)
    y = df['close']

    n = len(X)
    X_train, X_test, X_val = X.iloc[0:int(n*0.7), :], X.iloc[int(n*0.7):int(n*0.9),:], X.iloc[int(n*0.9):, :]
    y_train, y_test, y_val = y[0:int(n*0.7)], y[int(n*0.7):int(n*0.9)], y[int(n*0.9):]


    return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
def ARIMA_testing(df):

    X_train, X_test, X_val, y_train, y_test, y_val = divides_into_samples_2(df)

    # Поиск лучшей модели
    best_model = auto_arima(y_train, start_p=0, start_q=0, max_p=7, max_q=7, m=12, seasonal=False, stepwise=True, trace=True)

    # Извлечение параметров p, d и q
    p = best_model.order[0]
    d = best_model.order[1]
    q = best_model.order[2]

    model = sm.tsa.ARIMA(y_train, order=(p, d, q))
    model_fit = model.fit()

    # Предсказание на валидационной выборке
    y_pred_test = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)

    # Оценка точности на валидационной выборке
    mape_val = mean_absolute_percentage_error(y_test, y_pred_test)
    print(f"MAPE на валидационной выборке: {mape_val}")
    return y_pred_test, y_test, X_test

def make_arima_pred(some_sata):
    y_pred_test, y_test, X_test = ARIMA_testing(some_sata)

    X_test['close'] = list(y_test)
    X_test['pred'] = list(y_pred_test)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_test['newsDatetime'], y=X_test['close'], mode='lines', name='val'))
    fig.add_trace(go.Scatter(x=X_test['newsDatetime'], y=X_test['pred'], mode='lines', name='pred'))
    fig.update_layout(title='Визуализация индикатора', xaxis_title='Дата', yaxis_title='Значение')
    fig.show()

In [ ]:
make_arima_pred(bitcoin_data_clean)

In [ ]:
make_arima_pred(eth_data_clean)

In [ ]:
make_arima_pred(sol_data_clean)

In [ ]:
make_arima_pred(ripp_data_clean)

In [ ]:
make_arima_pred(card_data_clean)

In [ ]:
make_arima_pred(doge_data_clean)

# SARIMA

In [ ]:
def SARIMA_testing(df):
    y_pred_val = []

    X_train, X_test, X_val, y_train, y_test, y_val = divides_into_samples_2(df)

    # Указываем период сезонности
    seasonal_period = 12

    # Ищем лучшую модель SARIMA
    sarima_model = auto_arima(
      y_train,
      start_p=0, start_q=0, start_P=0, start_Q=0,
      max_p=7, max_q=7, max_P=3, max_Q=3,
      m=seasonal_period,
      seasonal=True,
      stepwise=True,
      trace=True
    )

    # Получаем подобранные параметры
    order = sarima_model.order
    seasonal_order = sarima_model.seasonal_order

    # Обучение модели SARIMA
    try:
        model = sm.tsa.statespace.SARIMAX(y_train, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False) # disp=False подавляет вывод процесса оптимизации

        # Предсказание на валидационной выборке
        y_pred_val = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1)

        # Оценка точности на валидационной выборке
        mape_val = mean_absolute_percentage_error(y_val, y_pred_val)
        print(f"MAPE на валидационной выборке: {mape_val}")

    except Exception as e:
        print(f"Ошибка при обучении модели SARIMAX: {e}")
    return y_pred_val, y_val, X_val

In [ ]:
def make_sarima_pred(some_data):
    y_pred_val, y_val, X_val = SARIMA_testing(some_data)

    X_val['close'] = list(y_val)
    X_val['pred'] = list(y_pred_val)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_val['newsDatetime'], y=X_val['close'], mode='lines', name='val'))
    fig.add_trace(go.Scatter(x=X_val['newsDatetime'], y=X_val['pred'], mode='lines', name='pred'))
    fig.update_layout(title=f'Визуализация индикатора', xaxis_title='Дата', yaxis_title='Значение')
    fig.show()

In [ ]:
make_sarima_pred(eth_data_clean)

In [ ]:
make_sarima_pred(bitcoin_data_clean)

In [ ]:
make_sarima_pred(sol_data_clean)

In [ ]:
make_sarima_pred(doge_data_clean)

In [ ]:
make_sarima_pred(ripp_data_clean)

In [ ]:
make_sarima_pred(card_data_clean)

# Сравнение двух временных рядов

In [ ]:
## корреляционнная матрица
def draw_corr_matr(some_data):
    corr_matr = some_data[['diff', 'negative', 'positive', 'important', 'liked',
       'disliked', 'comments', 'close']].corr()
    sns.heatmap(corr_matr, annot=True, cmap='coolwarm')

In [ ]:
draw_corr_matr(eth_data_clean)

In [ ]:
draw_corr_matr(bitcoin_data_clean)

In [ ]:
draw_corr_matr(sol_data_clean)

In [ ]:
draw_corr_matr(doge_data_clean)

In [ ]:
draw_corr_matr(ripp_data_clean)

In [ ]:
draw_corr_matr(card_data_clean)

In [ ]:
# Применение теста Колмогорова-Смирнова для первого датасета
def kolm_smirnov(some_data):
    time = some_data['newsDatetime']
    ## найдем соотношение цен для двух подряд идущих дней
    series1 = some_data['close'].diff() / (some_data['close'].diff() + some_data['close'])
    series1 = (series1 - series1.mean()) / series1.std()
    series1.index = some_data['newsDatetime']

    series2 = (some_data['diff'] - some_data['diff'].mean()) / some_data['diff'].std()
    series2.index = some_data['newsDatetime']

    #series3 = some_data2[]

    ks_stat, p_value = ks_2samp(series1, series2)

    # Вывод результатов
    print(f"Статистика теста Колмогорова-Смирнова: {ks_stat}")
    print(f"p-value: {p_value}")

    # Интерпретация результатов
    alpha = 0.05
    if p_value < alpha:
        print("Распределения двух временных рядов статистически различны.")
    else:
        print("Невозможно отвергнуть гипотезу, что распределения двух временных рядов одинаковы.")

    # Визуализация данных для сравнения
    plt.figure(figsize=(10, 6))
    plt.plot(time, series1, label="Series 1", linestyle='-', color='blue')
    plt.plot(time, series2, label="Series 2", linestyle='--', color='red')
    plt.title("Сравнение двух временных рядов")
    plt.xlabel("Время")
    plt.ylabel("Значение")
    plt.legend()
    plt.show()

In [ ]:
bitcoin_data_clean

In [ ]:
kolm_smirnov(bitcoin_data_clean)

In [ ]:
kolm_smirnov(eth_data_clean)

In [ ]:
kolm_smirnov(sol_data_clean)

In [ ]:
kolm_smirnov(doge_data_clean)

In [ ]:
kolm_smirnov(ripp_data_clean)

In [ ]:
kolm_smirnov(card_data_clean)

In [ ]:
bitcoin_data_new

In [ ]:
# Проверка стационарности
def check_stationarity(series):
    result = adfuller(series.dropna())
    p_value = result[1]
    return p_value < 0.05

## Tест Грейнджера (для второго датасета)
def granger_test(some_data, max_lag=4):

    series1 = some_data['close'].diff() / (some_data['close'].diff() + some_data['close'])
    series1 = (series1 - series1.mean()) / series1.std()
    series1.index = some_data['date']
    # Проверка стационарности
    if not check_stationarity(series1):
        print('ряд не стационарен')
        return


    series2 = (some_data['mean_index'] - some_data['mean_index'].mean()) / some_data['mean_index'].std()
    series2.index = some_data['date']
    if not check_stationarity(series2):
        print('ряд не стационарен')
        return

    time = some_data['date']

    test_data = pd.concat([series1, series2], axis=1).iloc[1:]
    results = grangercausalitytests(test_data, maxlag=max_lag, verbose=False)

    print(f"Результаты для лагов 1-{max_lag}:")
    for lag in range(1, max_lag+1):
        p_value = results[lag][0]['ssr_ftest'][1]
        print(f"Лаг {lag}: p-value = {p_value:.4f}",
              "*Отвергаем H0*" if p_value < 0.05 else "")





    # Визуализация данных для сравнения
    plt.figure(figsize=(10, 6))
    plt.plot(time, series1, label="Series 1", linestyle='-', color='blue')
    plt.plot(time, series2, label="Series 2", linestyle='--', color='red')
    plt.title("Сравнение двух временных рядов")
    plt.xlabel("Время")
    plt.ylabel("Значение")
    plt.legend()
    plt.show()


In [ ]:
granger_test(bitcoin_data_new, max_lag=4)

In [ ]:
granger_test(eth_data_new, max_lag=4)